In [4]:
from maraboupy import Marabou, MarabouCore
import pandas as pd

In [5]:
import os
import csv
import numpy as np


def write_values_to_csv(values, filename, caller_file_path):
    """
    Write values to a csv file in the directory of the caller file.

    Args:
        values: The values to be written.
        filename: The name of the file to be written.
        caller_file_path: The file path of the caller.

    Returns:
        None
    """
    # Get the directory of the caller file
    caller_directory = os.path.dirname(os.path.abspath(caller_file_path))
    full_path = os.path.join(caller_directory, filename)

    with open(full_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Check if values is a dictionary or numpy array and convert to list
        if isinstance(values, dict):
            values = list(values.values())
        elif isinstance(values, np.ndarray):
            values = values.tolist()

        writer.writerow(values)

    print(f"Values written to {full_path}")




def set_input_range(network, inputVars, mean_values, initial_range):
    for i, mean_val in enumerate(mean_values):
        network.setLowerBound(inputVars[i], mean_val - initial_range[i])
        network.setUpperBound(inputVars[i], mean_val + initial_range[i])


def define_output_conditions(network, outputVars, desired_output_class):
    for i in range(len(outputVars)):
        if i != desired_output_class:
            network.addInequality([outputVars[0][i], outputVars[0][desired_output_class]], [1, -1], 0)


# @debug
def check_results(status, initial_range, step_size):
    """
    check the results of the verification

    Args:
        status: the status of the verification
        initial_range: the initial range of the input
        step_size: the step size of the input

    Returns:
        unsat: whether the verification is unsat
        initial_range: the updated initial range
    """
    if status == "unsat":
        for i in range(len(initial_range)):
            initial_range[i] += step_size[i]
        return True, initial_range
    else:
        return False, initial_range



def block_solution(network, values, inputVars):
    """
    Adds constraints to the network to block the current solution.

    Args:
    network (MarabouNetwork): The Marabou network object.
    values (dict): The current solution values.
    inputVars (list): List of input variables.
    """
    for var in inputVars:
        # Create a constraint that the variable is less than the current solution
        eq1 = MarabouCore.Equation(MarabouCore.Equation.LE)
        eq1.addAddend(1, var)
        eq1.setScalar(values[var.item()] - 0.05)

        # Create a constraint that the variable is greater than the current solution
        eq2 = MarabouCore.Equation(MarabouCore.Equation.GE)
        eq2.addAddend(1, var)
        eq2.setScalar(values[var.item()] + 0.05)

        # Add the disjunction of the two constraints to the network
        network.addDisjunctionConstraint([[eq1], [eq2]])

In [10]:
file_name = '/home/adam/PycharmProjects/FurtherResearch/Model/Top_LDA/exo_model_top10_without_softmax.onnx'
network = Marabou.read_onnx(file_name)

inputVars = network.inputVars[0][0]
outputVars = network.outputVars[0]
print(inputVars)
print(outputVars)

[0 1 2 3 4 5 6 7 8 9]
[[10 11 12]]


In [ ]:
    high_fatigue = pd.read_csv(
        '/home/adam/FurtherResearch/Experiments/Top_LDA/dataset_statistics_mapped_fatigue_level_high.csv')
    mean_values = high_fatigue['mean'][:-1].values

    initial_range = [0.01] * 63
    step_size = high_fatigue['std'][:-1].values * 0.1

    options = Marabou.createOptions(numWorkers=20, initialTimeout=5, initialSplits=100, onlineSplits=100,
                                    timeoutInSeconds=1800, timeoutFactor=1.5,
                                    verbosity=2, snc=True, splittingStrategy='auto',
                                    sncSplittingStrategy='auto', restoreTreeStates=False,
                                    splitThreshold=20, solveWithMILP=True, dumpBounds=False)
    sat_counter = 0  # Initialize sat counter
    unsat = True
    while unsat:
        network = Marabou.read_onnx(file_name)
        set_input_range(network, inputVars, mean_values, initial_range)
        define_output_conditions(network, outputVars, 2)
        result = network.solve(verbose=True, options=options)
        status, values, stats = result
        unsat, initial_range = check_results(status, initial_range, step_size)
        if status == "sat":
            print("Solution found!")
        elif status == "unsat":
            print("No solution found.")
            print("Time:", stats.getTotalTimeInMicro())

    range = [ir - ss for ir, ss in zip(initial_range, step_size)]
    print(f"最小 UNSAT 范围: {range}")
    write_values_to_csv(range, 'top10_range.csv', __file__)
    write_values_to_csv(values, 'top10_values.csv', __file__)

